In [1]:
class Camera(IdealCamera): ###noisesim_occlusion### 
    def __init__(self, env_map,
                 distance_range=(0.5, 350),
                 direction_range=(-math.pi, math.pi),
                 distance_noise_rate=0.05, direction_noise=math.pi/180,
                 distance_bias_rate_stddev=0.05, direction_bias_stddev=math.pi/180,
                 phantom_prob=0.0, phantom_range_x=(-5.0,5.0), phantom_range_y=(-5.0,5.0),
                 oversight_prob=0.1, occlusion_prob=0.0): #occlusion_prob追加
        super().__init__(env_map, distance_range, direction_range)
        
        self.distance_noise_rate = distance_noise_rate
        self.direction_noise = direction_noise  
        self.distance_bias_rate_std = norm.rvs(scale=distance_bias_rate_stddev)
        self.direction_bias = norm.rvs(scale=direction_bias_stddev) 
        
        rx, ry = phantom_range_x, phantom_range_y
        self.phantom_dist = uniform(loc=(rx[0], ry[0]), scale=(rx[1]-rx[0], ry[1]-ry[0]))
        self.phantom_prob = phantom_prob
        
        self.oversight_prob = oversight_prob
        self.occlusion_prob = occlusion_prob #追加
        
        self.landmark_to_goal = 30
        
    def noise(self, relpos):  
        ell = norm.rvs(loc=relpos[0], scale=relpos[0]*self.distance_noise_rate)
        phi = norm.rvs(loc=relpos[1], scale=self.direction_noise)
        return np.array([ell, phi]).T
    
    def bias(self, relpos): 
        return relpos + np.array([relpos[0]*self.distance_bias_rate_std,
                                  self.direction_bias]).T
    
    def data(self, cam_pose):
        observed = []

        for lm in self.map.objects:
            #print("camera data obj=",lm.pos)
            #print("camera data cam=", cam_pose)
            z = self.observation_function(cam_pose, lm.pos)
#             z = self.phantom(cam_pose, z) 
#             z = self.occlusion(z) #追加
#             z = self.oversight(z)
            #print("a", self.visible(z))
            if self.visible(z):
                #print("b")
                z = self.bias(z)
                z = self.noise(z)
                observed.append((z, lm.id))         
        self.lastdata = observed
        return observed
    
    def angle(self, cam_pose):
        observed = []
        for lm in self.map.objects:
            t = lm.pos[2]
            delta = 90 / 180 * math.pi
            goal_pose = lm.pos + np.array([ self.landmark_to_goal * math.cos(t + delta),
                                           self.landmark_to_goal * math.cos(t + delta),
                                           delta]) 
            angle = self.observation_function(cam_pose, goal_pose)
            observed.append(angle)
        return observed

In [ ]:
time_interval = 0.1
world = World(1300, time_interval, debug = False)

m = Map()
data1 = [[0.15,0,200],[1.0, -3.6 * math.pi / 180, 25],[0.15,0,400],[0, -3.6 * math.pi / 180, 25],[0.15,0,300],[1.0, -3.6 * math.pi / 180,25],[0.15,0,300],]
agent1 = AgentY(0.1,data1)
initial_pose_1 = np.array([-100,-100,0]).T
m.append_object(Landmark(initial_pose_1, agent = agent1))
world.append(m)
Camera(m).angle()